In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
#mount google drive path
from google.colab import drive
drive.mount('/drive')
 
 #common header - to avoid bot detection from websites
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}
 #varied request header
headers1 = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
        'referrer': 'https://google.com',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Pragma': 'no-cache',
    }

#web_link='https://www.fiverr.com/search/gigs?query=market%20research&source=top-bar&acmpl=1&search_in=everywhere&search-autocomplete-original-term=&search-autocomplete-available=true&search-autocomplete-type=recent-gigs-suggest&search-autocomplete-position=0'
web_link='https://www.fiverr.com/search/gigs?query=market%20research&pos=1&context=recommendation&context_alg=related_search_terms&source=related_search_terms&filter=rating'
urls=requests.get(web_link, headers=headers1)
 
#soup object creation from url content. lxml is an html parser. Variants are html.parser, lxml, html5 
soup=BeautifulSoup(urls.content, 'lxml')

In [ ]:
print(soup)

![alt text](https://www.fiverr.com/search/gigs?query=market%20research&pos=1&context=recommendation&context_alg=related_search_terms&source=related_search_terms&filter=rating)


Fiver page screenshot

In [ ]:
 
mylist=[]
 
 #iterating through all the tags with gig-wrapper-card. In fiverr
for seller_parent in soup.find_all(name='div', class_='gig-wrapper card'):
 
  list1=[]
  #each gig has a unique gig ID
  id=seller_parent.attrs['data-gig-id']
  #name-object contains name, seller-level and seller level hint
  seller_name_object=seller_parent.find('span', class_='seller-name')
  name=seller_name_object.a.text[3:]
  #catches certain excemptions where seller hint and seller level is empty for gigs
  try:
    seller_hint=seller_name_object.find_all('span')[1].attrs['data-hint']
  except IndexError:
    seller_hint='None'
  try:
    seller_level=seller_name_object.find_all('span')[1].text
  except:
    seller_level=0
  #contains seller link description
  link_description=(seller_parent.find('h3').text)
  #link to gig 
  link=('https://fiver.com'+seller_parent.find('h3').a.attrs['href'])
  #ratings in format  avg ratings (total_ratings)
  ratings=seller_parent.find('span', class_='gig-rating text-body-2').text
  #starting price for the gig
  starting_price=seller_parent.footer.find('a', class_='price')['title']
  list1=[id, name, ratings, seller_level, starting_price, link, link_description, seller_hint]
  print(list1)
  mylist.append(list1)

['151430837_1', 'vesgaje', '5.0(15)', 0, '$65', 'https://fiver.com/vesgaje/deliver-valuable-market-research-comprehensive-and-concise?context_referrer=search_gigs&source=related_search_terms&ref_ctx_id=772bc0f7-d7af-405d-a120-fcf0ccee80c0&pckg_id=1&pos=2', 'I will deliver valuable market research, comprehensive and concise', 'None']
['114064784_2', 'sasha_quality', '5.0(317)', 'Level 2 Seller', '$20', 'https://fiver.com/sasha_quality/assist-in-internet-research-and-writing?context_referrer=search_gigs&source=related_search_terms&ref_ctx_id=772bc0f7-d7af-405d-a120-fcf0ccee80c0&pckg_id=1&pos=3', 'I will assist in online, market research and business research', 'Completed at least 50 orders on time with a minimum 4.8 rating']
['147613610_3', 'mary_proarticle', '5.0(28)', 0, '$60', 'https://fiver.com/mary_proarticle/build-comprehensive-market-research-competitor-analysis-business-plan-web-report?context_referrer=search_gigs&source=related_search_terms&ref_ctx_id=772bc0f7-d7af-405d-a120-fcf

Headers: Id, Name, rating, seller_level, starting_price ,Link,link_description, seller_hint

In [ ]:

#column name headers for the dataframe to be created
column_names=['id','name','rating','seller_level','starting_price','link','link_description','seller_hint']
df=pd.DataFrame(mylist, columns=column_names)

print(df)
#saves dataframe to googledrive link given. Appends if file already exists
df.to_csv('/drive/My Drive/Colab Notebooks/foo.csv', mode='a', header=False)

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
            id  ...                                        seller_hint
0  151430837_1  ...                                               None
1  114064784_2  ...  Completed at least 50 orders on time with a mi...
2  147613610_3  ...                                               None
3  162665799_4  ...                                                   
4  135981233_5  ...  Completed at least 50 orders on time with a mi...
5   12196674_7  ...  Vetted by Fiverr and completed at least 100 or...

[6 rows x 8 columns]
